In [ ]:
import os
os.kill(os.getpid(), 9)

In [1]:
!pip install --upgrade gensim

  Obtaining dependency information for gensim from https://files.pythonhosted.org/packages/79/7b/747fcb06280764cf20353361162eff68c6b0a3be34c43ead5ae393d3b18e/gensim-4.3.3-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for numpy<2.0,>=1.18.5 from https://files.pythonhosted.org/packages/16/2e/86f24451c2d530c88daf997cb8d6ac622c1d40d19f5a031ed68a4b73a374/numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
import requests
from time import sleep
import random
import numpy as np
#
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
#
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
import gensim.downloader as api

In [13]:
# incarc word-vectors pretrained din gensim-data
word_vectors = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [25]:
def get_embedding(word):
    return torch.tensor(word_vectors[word]) if word in word_vectors else torch.zeros(word_vectors.vector_size)

In [96]:
print(get_embedding('leaf').size())
print(get_embedding('leaf'))

torch.Size([100])
tensor([-0.9023,  1.4086, -0.8489, -0.5312,  0.2260,  0.1224, -0.1577, -0.0540,
        -0.7410, -0.5026, -0.3678,  0.5828,  0.3974,  0.4491, -0.2141,  0.5729,
         0.4034, -0.2570,  1.1080, -0.4805, -0.5136, -0.2244, -0.4224,  0.7832,
         0.7043,  1.3723,  0.4973, -0.0573,  0.1240,  0.1095,  0.4125,  0.6970,
         1.0663, -0.7788, -0.0498,  0.4541, -0.3222,  0.4862,  0.1286, -0.2822,
         0.2708, -0.9428, -0.0398, -0.3912,  0.1269,  0.5623,  0.3998, -0.0591,
        -0.4011,  0.1073,  0.1584, -0.6320, -0.9100,  0.8339,  0.0903, -0.4712,
        -0.7235,  0.4057,  1.5603, -0.0701,  0.2409,  0.9905, -0.1014,  0.6065,
         0.9442, -0.6140,  0.7192, -0.1301, -0.1404, -0.3287, -0.5289,  0.0599,
        -0.3051, -0.1665,  0.3543,  0.1868, -0.1084,  0.4288,  0.0197,  0.6008,
        -0.4965,  0.0702, -1.3217,  0.0520,  0.3516, -0.5940,  0.6948, -0.2726,
         0.4192,  0.7808, -0.3835, -0.4826, -0.6205,  0.2504, -0.3814, -0.3273,
        -0.8350, -0.35

In [53]:
host = "http://172.18.4.158:8000"
post_url = f"{host}/submit-word"
get_url = f"{host}/get-word"
status_url = f"{host}/status"

NUM_ROUNDS = 5

predefined_words = {
    k: {"text": v["text"].lower(), "cost": v["cost"]} for k, v in {
      1: {"text": "Feather", "cost": 1},
      2: {"text": "Coal", "cost": 1},
      3: {"text": "Pebble", "cost": 1},
      4: {"text": "Leaf", "cost": 2},
      5: {"text": "Paper", "cost": 2},
      6: {"text": "Rock", "cost": 2},
      7: {"text": "Water", "cost": 3},
      8: {"text": "Twig", "cost": 3},
      9: {"text": "Sword", "cost": 4},
      10: {"text": "Shield", "cost": 4},
      11: {"text": "Gun", "cost": 5},
      12: {"text": "Flame", "cost": 5},
      13: {"text": "Rope", "cost": 5},
      14: {"text": "Disease", "cost": 6},
      15: {"text": "Cure", "cost": 6},
      16: {"text": "Bacteria", "cost": 6},
      17: {"text": "Shadow", "cost": 7},
      18: {"text": "Light", "cost": 7},
      19: {"text": "Virus", "cost": 7},
      20: {"text": "Sound", "cost": 8},
      21: {"text": "Time", "cost": 8},
      22: {"text": "Fate", "cost": 8},
      23: {"text": "Earthquake", "cost": 9},
      24: {"text": "Storm", "cost": 9},
      25: {"text": "Vaccine", "cost": 9},
      26: {"text": "Logic", "cost": 10},
      27: {"text": "Gravity", "cost": 10},
      28: {"text": "Robots", "cost": 10},
      29: {"text": "Stone", "cost": 11},
      30: {"text": "Echo", "cost": 11},
      31: {"text": "Thunder", "cost": 12},
      32: {"text": "Karma", "cost": 12},
      33: {"text": "Wind", "cost": 13},
      34: {"text": "Ice", "cost": 13},
      35: {"text": "Sandstorm", "cost": 13},
      36: {"text": "Laser", "cost": 14},
      37: {"text": "Magma", "cost": 14},
      38: {"text": "Peace", "cost": 14},
      39: {"text": "Explosion", "cost": 15},
      40: {"text": "War", "cost": 15},
      41: {"text": "Enlightenment", "cost": 15},
      42: {"text": "Nuclear Bomb", "cost": 16},
      43: {"text": "Volcano", "cost": 16},
      44: {"text": "Whale", "cost": 17},
      45: {"text": "Earth", "cost": 17},
      46: {"text": "Moon", "cost": 17},
      47: {"text": "Star", "cost": 18},
      48: {"text": "Tsunami", "cost": 18},
      49: {"text": "Supernova", "cost": 19},
      50: {"text": "Antimatter", "cost": 19},
      51: {"text": "Plague", "cost": 20},
      52: {"text": "Rebirth", "cost": 20},
      53: {"text": "Tectonic Shift", "cost": 21},
      54: {"text": "Gamma-Ray Burst", "cost": 22},
      55: {"text": "Human Spirit", "cost": 23},
      56: {"text": "Apocalyptic Meteor", "cost": 24},
      57: {"text": "Earth’s Core", "cost": 25},
      58: {"text": "Neutron Star", "cost": 26},
      59: {"text": "Supermassive Black Hole", "cost": 35},
      60: {"text": "Entropy", "cost": 45},
    }.items()
}

In [54]:
for key, word_info in predefined_words.items():
  # print(word_info["text"])
  if get_embedding(word_info["text"]).sum() == 0:
    print(key, word_info)

42 {'text': 'nuclear bomb', 'cost': 16}
53 {'text': 'tectonic shift', 'cost': 21}
54 {'text': 'gamma-ray burst', 'cost': 22}
55 {'text': 'human spirit', 'cost': 23}
56 {'text': 'apocalyptic meteor', 'cost': 24}
57 {'text': 'earth’s core', 'cost': 25}
58 {'text': 'neutron star', 'cost': 26}
59 {'text': 'supermassive black hole', 'cost': 35}


In [55]:
predefined_words[42] = {"text": "bomb", "cost": 16}
predefined_words[53] = {"text": "earthquake", "cost": 21}
predefined_words[54] = {"text": "radiation", "cost": 22}
predefined_words[55] = {"text": "spirit", "cost": 23}
predefined_words[56] = {"text": "apocalypse", "cost": 24}
predefined_words[57] = {"text": "magma", "cost": 25}
predefined_words[58] = {"text": "neutron", "cost": 26}
predefined_words[59] = {"text": "blackhole", "cost": 35}

In [56]:
for key, word_info in predefined_words.items():
  # print(word_info["text"])
  if get_embedding(word_info["text"]).sum() == 0:
    print(key, word_info)

In [87]:
class StrengthModel(nn.Module):
    def __init__(self, embedding_dim):
        super(StrengthModel, self).__init__()
        self.fc1 = nn.Linear(embedding_dim * 2, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 1)

    def forward(self, word_embedding, challenge_embedding):
        word_embedding = word_embedding.unsqueeze(0) if word_embedding.dim() == 1 else word_embedding
        challenge_embedding = challenge_embedding.unsqueeze(0) if challenge_embedding.dim() == 1 else challenge_embedding
        x = torch.cat((word_embedding, challenge_embedding), dim=1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [58]:
class StrengthDataset(Dataset):
    def __init__(self, pairs, labels):
        self.pairs = pairs
        self.labels = labels

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        return self.pairs[idx], self.labels[idx]

In [73]:
def train_model(model, dataset, epochs=10, lr=0.001):
    dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        total_loss = 0
        for (word_embedding, challenge_embedding), label in dataloader:
            optimizer.zero_grad()
            output = model(word_embedding, challenge_embedding).squeeze()
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

In [ ]:
predefined_words_texts = ['feather', 'coal', 'pebble', 'leaf', 'paper', 'rock', 'water', 'twig', 'sword', 'shield', 'gun', 'flame', 'rope', 'disease', 'cure', 'bacteria', 'shadow', 'light', 'virus', 'sound', 'time', 'fate', 'earthquake', 'storm', 'vaccine', 'logic', 'gravity', 'robots', 'stone', 'echo', 'thunder', 'karma', 'wind', 'ice', 'sandstorm', 'laser', 'magma', 'peace', 'explosion', 'war', 'enlightenment', 'bomb', 'volcano', 'whale', 'earth', 'moon', 'star', 'tsunami', 'supernova', 'antimatter', 'plague', 'rebirth', 'earthquake', 'radiation', 'spirit', 'apocalypse', 'magma', 'neutron', 'blackhole', 'entropy']

alive_creatures_challenges = ['man', 'snake', 'flower', 'plant', 'tree', 'fish', 'woman', 'person', 'bear', 'dog', 'cat']
easily_breakable_material_challenges = ['glass', 'phone', 'laptop', 'mouse', 'plate', 'wheel', 'car']
hardly_breakable_material_challenges = ['house', 'building', 'plane', 'platform', 'field']
imaterial_challenges = ['thought', 'smile', 'day', 'week', 'minute']
disease_challenges = ['bacteria', 'virus']

unharmful_predefined = ['feather', 'coal', 'leaf', 'paper', 'shield', 'rope', 'sound', 'time', 'fate', 'logic', 'echo', 'karma', 'wind', 'rebirth', 'spirit', 'light', 'peace', 'shadow', 'earth', 'moon', '']
disaster_predefined = ['apocalypse', 'magma', 'neutron', 'blackhole', 'entropy', 'earthquake', 'radiation', 'sandstorm', 'storm', 'gravity', 'thunder', 'explosion', 'war', 'volcano']
harmful_to_alive = ['sword', 'gun', 'flame', 'disease', 'bacteria', 'virus', 'robots', 'stone', 'water', 'rock', ]
mildly_harmful = ['pebble', 'twig']
beats_disease = ['cure', 'vaccine']


# for word1 in list1:
#     for word2 in list2:
#         if word1 in 

# (-100, 100) - range in care cuvintele nu sunt comparabile

# training data

# pairs = []
# labels = []

# for word1, word2 in all_combinations:
#     pairs.append((get_embedding(word1), get_embedding(word2)))

# labels = [
#     torch.tensor([150.0]),
#     # add your data
# ]

# dataset = StrengthDataset(pairs, labels)

for word in predefined_words_texts:
    if word not in unharmful_predefined and word not in disaster_predefined and word not in harmful_to_alive and word not in mildly_harmful and word not in beats_disease:
        print(word)

ice
laser
enlightenment
bomb
whale
star
tsunami
supernova
antimatter
plague


In [116]:
# definire si antrenare model

embedding_dim = word_vectors.vector_size
model = StrengthModel(embedding_dim)

train_model(model, dataset, epochs=100, lr=0.001)

Epoch 1, Loss: 1838635.2812
Epoch 2, Loss: 1837135.4062
Epoch 3, Loss: 1835504.5625
Epoch 4, Loss: 1833659.5312
Epoch 5, Loss: 1831607.1250
Epoch 6, Loss: 1829167.1250
Epoch 7, Loss: 1826484.8438
Epoch 8, Loss: 1823156.9688
Epoch 9, Loss: 1819441.4375
Epoch 10, Loss: 1815390.6875
Epoch 11, Loss: 1810546.0000
Epoch 12, Loss: 1806086.7812
Epoch 13, Loss: 1800206.1562
Epoch 14, Loss: 1794639.0938
Epoch 15, Loss: 1788365.2188
Epoch 16, Loss: 1780820.4062
Epoch 17, Loss: 1772655.8750
Epoch 18, Loss: 1764992.5625
Epoch 19, Loss: 1755802.0000
Epoch 20, Loss: 1747732.4688
Epoch 21, Loss: 1736799.1875
Epoch 22, Loss: 1728490.0000
Epoch 23, Loss: 1717823.0625
Epoch 24, Loss: 1707028.7500
Epoch 25, Loss: 1697089.5938
Epoch 26, Loss: 1682241.0625
Epoch 27, Loss: 1672054.1562
Epoch 28, Loss: 1659527.1875
Epoch 29, Loss: 1642446.4688
Epoch 30, Loss: 1633078.8750
Epoch 31, Loss: 1622246.5469
Epoch 32, Loss: 1609667.2812
Epoch 33, Loss: 1593205.7500
Epoch 34, Loss: 1580143.1250
Epoch 35, Loss: 1565896

In [123]:
def what_beats(challenge_word):
    challenge_embedding = get_embedding(challenge_word)

    best_word = None
    best_cost = float('inf')

    for word_info in predefined_words.values():
      word = word_info["text"]
      cost = word_info["cost"]
      word_embedding = get_embedding(word)

      strength = model(word_embedding, challenge_embedding).item()

      if strength > 100 and cost < best_cost:
            best_cost = cost
            best_word = word

    return best_word

def play_game(player_id):

    for round_id in range(1, NUM_ROUNDS+1):
        round_num = -1
        while round_num != round_id:
            response = requests.get(get_url)
            print(response.json())
            sys_word = response.json()['word'].lower()
            round_num = response.json()['round']

            sleep(1)

        if round_id > 1:
            status = requests.get(status_url)
            print(status.json())

        choosen_word, word_strength = what_beats(sys_word)
        data = {"player_id": player_id, "word_id": choosen_word, "round_id": round_id}
        response = requests.post(post_url, json=data)
        print(response.json())

In [124]:
print(what_beats("lion"))

feather
